# Init

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_key",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}

# Read from Bronze Layer

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

# Data Transformations 


In [0]:
df.display()

## Trim strings

In [0]:

for field in df.schema.fields:
    if isinstance(field.dataType, StringType): 
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
df.display()

## Normalize martial status
## Normalize gender

In [0]:
df = (
    df
    .withColumn(
        "cst_marital_status",
        f.when(f.upper(f.col("cst_marital_status")) == "S", "Single")
         .when(f.upper(f.col("cst_marital_status")) == "M", "Married")
         .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        f.when(f.upper(f.col("cst_gndr")) == "F", "Female")
         .when(f.upper(f.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )
)


In [0]:
df.display()

## Rename columns

In [0]:
for old_name, new_name in RENAME_MAP.items():
  df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.display()

# Write into Silver table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("silver.crm_customers")